# TVM：Python 调用 C++

下面逐步揭开 TVM 中 C++/C 与 Python 交互的机制。

在 C++ 中定义加法算子：

````{tab-set-code}
```{literalinclude} cpp/sym_add/src/sym_add.cc
:language: c++
```

```{literalinclude} cpp/sym_add/Makefile
:language: Makefile
```
````

## Python 端加载 C++ 端动态库

可以使用 {mod}`ctypes` 加载动态库：

In [1]:
import ctypes

# 作为全局加载，使全局外部符号对其他 dll 可见。
_LIB = ctypes.CDLL("outputs/libs/libtvm_ext.so", ctypes.RTLD_GLOBAL)

OSError: outputs/libs/libtvm_ext.so: undefined symbol: _ZNK3tvm7runtime6Object11DerivedFromEj

加载失败，是由于 `libtvm_ext.so` 是在 `libtvm.so` 基础上拓展的，故而需要先提前加载 `libtvm.so`，或者直接 `import tvm`：

In [2]:
import tvm
import ctypes

# 作为全局加载，使全局外部符号对其他 dll 可见。
_LIB = ctypes.CDLL("outputs/libs/libtvm_ext.so", ctypes.RTLD_GLOBAL)

加载动态库，也可以直接使用 {func}`~tvm_book.tvm_ext.libinfo._load_lib`：

In [1]:
from tvm_book.tvm_ext.libinfo import _load_lib

_LIB, _LIB_NAME = _load_lib() # 加载 libtvm_ext.so
_LIB_EXT, _LIB_EXT_NAME = _load_lib(name="libtvm_ext.so", search_path=["outputs/libs"])

回调 C++ 函数：

In [3]:
import tvm
sym_add = tvm.get_global_func("tvm_ext.sym_add")

测试：

In [4]:
from tvm import te
a = te.var("x")
b = te.var("y")
c = sym_add(a, b)
assert c.a == a and c.b == b
print(c)

x + y


这些调用细节可以借助 FFI 机制进行隐藏。

## 使用 {func}`tvm._ffi._init_api` 管理 TVM 插件

In [1]:
from tvm_book.tvm_ext.libinfo import _load_lib

_LIB_EXT, _LIB_EXT_NAME = _load_lib(name="libtvm_ext.so", search_path=["outputs/libs"])

In [2]:
import tvm

tvm._ffi._init_api("tvm_ext", __name__)

下面便可以直接使用 `tvm_ext` 下的函数了：

In [4]:
sym_add

```{tip}
C++ 端调用 Python 程序的示例请移步 [C++ 部署](../../tutorials/deploy/cpp)。
```

## 其他 C++ 打包函数的例子

### 加法偏函数

```c++
#include <tvm/runtime/packed_func.h>
#include <tvm/runtime/registry.h>

using namespace tvm::runtime;

namespace tvm_ext {
TVM_REGISTER_GLOBAL("tvm_ext.bind_add").set_body([](TVMArgs args_, TVMRetValue* rv_) {
  PackedFunc pf = args_[0];
  int b = args_[1];
  *rv_ = PackedFunc([pf, b](TVMArgs args, TVMRetValue* rv) { *rv = pf(b, args[0]); });
});

} // namespace tvm_ext

```

In [2]:
import tvm
from tvm_book.tvm_ext.libinfo import _load_lib

_LIB_EXT, _LIB_EXT_NAME = _load_lib(name="libtvm_ext.so", search_path=["outputs/libs"])
tvm._ffi._init_api("tvm_ext", __name__)

In [4]:
bind_add

In [5]:
def add(a, b):
    return a + b

f = bind_add(add, 7)
assert f(2) == 9

### C++ 外部设备的例子

```c++
#include <tvm/runtime/device_api.h>
#include <tvm/runtime/packed_func.h>
#include <tvm/runtime/registry.h>
// #include <tvm/tir/op.h>

using namespace tvm::runtime;

namespace tvm_ext {
TVM_REGISTER_GLOBAL("device_api.ext_dev").set_body([](TVMArgs args, TVMRetValue* rv) {
  *rv = (*tvm::runtime::Registry::Get("device_api.cpu"))();
});

} // namespace tvm_ext
```

In [1]:
import tvm
from tvm_book.tvm_ext.libinfo import _load_lib

_LIB_EXT, _LIB_EXT_NAME = _load_lib(name="libtvm_ext.so", search_path=["outputs/libs"])
tvm._ffi._init_api("tvm_ext", __name__)

In [2]:
import numpy as np
from tvm import te
n = 10
A = te.placeholder((n,), name="A")
B = te.compute((n,), lambda *i: A(*i) + 1.0, name="B")
s = te.create_schedule(B.op)

def check_llvm():
    f = tvm.build(s, [A, B], tvm.target.Target("ext_dev", "llvm"))
    dev = tvm.ext_dev(0)
    # launch the kernel.
    a = tvm.nd.array(np.random.uniform(size=n).astype(A.dtype), dev)
    b = tvm.nd.array(np.zeros(n, dtype=B.dtype), dev)
    f(a, b)
    np.testing.assert_allclose(b.numpy(), a.numpy() + 1)

check_llvm()

## 回调 C++ 端外部函数

```c++
#include <tvm/runtime/registry.h>

// 暴露给运行时的外部函数
extern "C" float TVMTestAddOne(float y) { return y + 1; }
```

In [1]:
import tvm
from tvm_book.tvm_ext.libinfo import _load_lib

_LIB_EXT, _LIB_EXT_NAME = _load_lib(name="libtvm_ext.so", search_path=["outputs/libs"])
tvm._ffi._init_api("tvm_ext", __name__)

In [4]:
import numpy as np
from tvm import te
n = 10
A = te.placeholder((n,), name="A")
B = te.compute(
    (n,), lambda *i: tvm.tir.call_extern("float32", "TVMTestAddOne", A(*i)), name="B"
)
s = te.create_schedule(B.op)

def check_llvm():
    f = tvm.build(s, [A, B], "llvm")
    dev = tvm.cpu(0)
    # launch the kernel.
    a = tvm.nd.array(np.random.uniform(size=n).astype(A.dtype), dev)
    b = tvm.nd.array(np.zeros(n, dtype=B.dtype), dev)
    f(a, b)
    np.testing.assert_allclose(b.numpy(), a.numpy() + 1)

check_llvm()

## 提取外部 C++ 函数

```c++
// 这个回调方法允许扩展，使 TVM 能够提取。
// 当想要使用仅包含头文件的最小版本的 TVM 运行时，这种方法会很有帮助。
extern "C" int TVMExtDeclare(TVMFunctionHandle pregister) {
  const PackedFunc& fregister = GetRef<PackedFunc>(static_cast<PackedFuncObj*>(pregister));
  // 等价于 const PackedFunc& fregister = *static_cast<PackedFunc*>(pregister);
  auto mul = [](TVMArgs args, TVMRetValue* rv) {
    int x = args[0];
    int y = args[1];
    *rv = x * y;
  };
  fregister("mul", PackedFunc(mul));
  return 0;
}
```

In [4]:
import tvm
from tvm_book.tvm_ext.libinfo import _load_lib

_LIB, _LIB_NAME = _load_lib(name="libtvm_ext.so", search_path=["outputs/libs"])
tvm._ffi._init_api("tvm_ext", __name__)

In [5]:
fdict = tvm._ffi.registry.extract_ext_funcs(_LIB.TVMExtDeclare)
assert fdict["mul"](3, 4) == 12